In [23]:
import numpy as np  # 수학/배열 연산용

BOARD_ROWS = 3      # 보드 행 개수
BOARD_COLS = 4      # 보드 열 개수

WIN_STATE = (0, 3)      # 승리 위치
LOSE_STATE = (1, 3)     # 패배 위치
BLOCKED_STATE = (1, 1)  # 이동 불가 칸
START = (2, 0)          # 시작 위치

DETERMINISTIC = False   # False면 행동 결과에 랜덤성 있음


In [24]:
class State:
    def __init__(self, state = START):
        self.state = state           # 현재 위치
        self.isEnd = False           # 게임 종료 여부
        self.determine = DETERMINISTIC  # 결정론적 이동 여부

    def giveReward(self):
        # 현재 위치에 따른 보상 반환
        if self.state == WIN_STATE:
            return 1
        elif self.state == LOSE_STATE:
            return -1
        else:
            return 0

    def isEndFunc(self):
        # 현재 위치가 승리/패배 상태면 종료 표시
        if (self.state == WIN_STATE) or (self.state == LOSE_STATE):
            self.isEnd = True

    def _chooseActionProb(self, action):
        # 확률적으로 행동 선택 (랜덤성)
        if action == "U":
            return np.random.choice(["U", "L", "R"], p=[0.8, 0.1, 0.1])
        if action == "D":
            return np.random.choice(["D", "L", "R"], p=[0.8, 0.1, 0.1])
        if action == "L":
            return np.random.choice(["L", "U", "D"], p=[0.8, 0.1, 0.1])
        if action == "R":
            return np.random.choice(["R", "U", "D"], p=[0.8, 0.1, 0.1])

    def nxtPosition(self, action):
        # 다음 위치 계산
        if self.determine:
            # 결정론적 이동
            if action == "U":
                nxtState = (self.state[0]-1, self.state[1])
            elif action == "D":
                nxtState = (self.state[0]+1, self.state[1])
            elif action == "L":
                nxtState = (self.state[0], self.state[1]-1)
            else:
                nxtState = (self.state[0], self.state[1]+1)
            self.determine = False
        else:
            # 확률적 이동
            action = self._chooseActionProb(action)
            self.determine = True
            nxtState = self.nxtPosition(action)

        # 이동 범위 확인 및 막힌 칸 처리
        if (0 <= nxtState[0] <= 2) and (0 <= nxtState[1] <= 3) and (nxtState != BLOCKED_STATE):
            return nxtState
        return self.state  # 이동 불가 시 현재 위치 유지


In [25]:
class Agent:

    def __init__(self):
        self.states = []               # 에이전트가 이동한 상태와 행동 기록
        self.actions = ["U", "D", "L", "R"]  # 가능한 행동
        self.State = State()            # 현재 상태(State 객체)
        self.isEnd = self.State.isEnd   # 현재 상태가 종료인지 여부
        self.lr = 0.2                   # 학습률(Learning Rate)
        self.decay_gamma = 0.9          # 할인율(Gamma)

        # Q-테이블 초기화: 모든 상태에서 모든 행동의 값 0으로 시작
        self.Q_values = {}
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                self.Q_values[(i, j)] = {}
                for a in self.actions:
                    self.Q_values[(i, j)][a] = 0

    def chooseAction(self):
        # 현재 상태에서 Q값이 가장 높은 행동 선택 (탐욕적)
        max_nxt_reward = 0
        action = ""
        for a in self.actions:
            current_position = self.State.state
            nxt_reward = self.Q_values[current_position][a]
            if nxt_reward >= max_nxt_reward:
                action = a
                max_nxt_reward = nxt_reward
        return action

    def takeAction(self, action):
        # 선택한 행동으로 이동 후 새로운 State 객체 반환
        position = self.State.nxtPosition(action)
        return State(state = position)

    def reset(self):
        # 에피소드 종료 후 초기화
        self.states = []
        self.State = State()
        self.isEnd = self.State.isEnd

    def play(self, episodes = 10):
        # Q-learning 학습 루프
        i = 0
        while i < episodes:
            if self.State.isEnd:
                # 종료 상태면 보상 부여 및 Q값 업데이트
                reward = self.State.giveReward()
                for a in self.actions:
                    self.Q_values[self.State.state][a] = reward
                for s in reversed(self.states):
                    current_q_value = self.Q_values[s[0]][s[1]]
                    reward = current_q_value + self.lr * (self.decay_gamma * reward - current_q_value)
                    self.Q_values[s[0]][s[1]] = round(reward, 3)
                self.reset()
                i += 1
            else:
                # 학습 중이면 행동 선택 후 이동
                action = self.chooseAction()
                self.states.append([(self.State.state), action])  # 상태-행동 기록
                self.State = self.takeAction(action)
                self.State.isEndFunc()  # 종료 여부 확인
                self.isEnd = self.State.isEnd


In [26]:
ag = Agent()        # 에이전트 객체 생성 (Q-table 초기화 등)

ag.play(1000)       # 1000 에피소드 동안 학습 실행
                     # - 매 에피소드마다 상태-행동 쌍을 기록
                     # - 종료 상태에서 보상 받고 Q-table 갱신

print("latest Q-values ... \n")
print(ag.Q_values)  # 학습이 끝난 후 모든 상태에서 행동별 Q값 출력

latest Q-values ... 

{(0, 0): {'U': 0, 'D': 0, 'L': 0, 'R': 0.582}, (0, 1): {'U': 0, 'D': 0, 'L': 0, 'R': 0.729}, (0, 2): {'U': 0, 'D': 0, 'L': 0, 'R': 0.684}, (0, 3): {'U': 1, 'D': 1, 'L': 1, 'R': 1}, (1, 0): {'U': 0, 'D': 0, 'L': 0, 'R': 0.245}, (1, 1): {'U': 0, 'D': 0, 'L': 0, 'R': 0}, (1, 2): {'U': 0, 'D': 0, 'L': 0.311, 'R': -0.18}, (1, 3): {'U': -1, 'D': -1, 'L': -1, 'R': -1}, (2, 0): {'U': 0, 'D': 0, 'L': 0.099, 'R': -0.001}, (2, 1): {'U': 0, 'D': 0, 'L': 0.099, 'R': -0.006}, (2, 2): {'U': 0, 'D': 0, 'L': 0.1, 'R': -0.031}, (2, 3): {'U': 0, 'D': 0, 'L': 0, 'R': -0.18}}
